In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
load_dotenv()

In [ ]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

In [ ]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 2, 1),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 2, 1), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((124, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation=LeakyReLU(alpha=0.01)))

model.summary()

In [ ]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/d-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val[:, 0]), epochs=3, callbacks=[cp_view])

In [ ]:

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/d-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val[:, 1]), epochs=3, callbacks=[cp_cart])

In [ ]:

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/d-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val[:, 2]), epochs=3, callbacks=[cp_remove_from_cart])

In [ ]:

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/d-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val[:, 3]), epochs=3, callbacks=[cp_purchase])